In [1]:
###################################################################
######  Various functions for working with toric varieties   ######
######  - From a list to an element of Pic(X) and back       ######
######  - Eff, Nef and Mov cones                             ######
######  - Check if a divisor is ample or movable             ######
######  - Check if the variety is a Fano variety             ######
######  - Secondary Fan (chambers of Eff)                    ######
######  Construct a toric variety from primitive relations   ######
######  Construct a toric projective bundle                  ######
###################################################################

In [2]:
from itertools import combinations

In [3]:
##################################################################
######  Find the image of divisors in Pic(X) and back       ######
##################################################################

def Pic(X,D):
    divisor = sum(D[i] * X.divisor(i) for i in range(len(D)))
    return X.rational_class_group()(divisor)
    
def unPic(X,D):
    A=D.lift()
    return [A[i][0] for i in range(len(X.fan().rays()))]
    
########################################################
######## Image in Pic X of invariant divisors  #########
########################################################

def invariant_in_pic(X):
    rays=X.fan().rays()
    diveff=[]
    for i in range(len(rays)):
        diveff.append(X.rational_class_group()(X.divisor(i)))
    return diveff
    
########################################################
################ Eff, Mov and Nef cones ################
########################################################

def eff_cone(X):
    L=X.Kaehler_cone().lattice()
    return Cone(invariant_in_pic(X),lattice=L)

def nef_cone(X):
    return X.Kaehler_cone()

def mov_cone(X):
    rays=X.fan().rays()
    movcones=[]
    L=X.Kaehler_cone().lattice()
    diveff=invariant_in_pic(X)
    for i in range(len(rays)):
        excluded_rays = [diveff[j] for j in range(len(diveff)) if j != i]
        cone = Cone(excluded_rays,lattice=L)
        movcones.append(cone)    
    intersection = movcones[0]
    for cone in movcones[1:]:
        intersection = intersection.intersection(cone)
    return intersection

###########################################
###   Compute the Picard number of X   ####
###########################################

def picard_number(X): 
    if 'Polyhedra' in str(X.category()):
        X = ToricVariety(NormalFan(X.affine_hull_projection()))
    return len(X.fan().rays())- X.dimension()    

###########################################################
###  Find an ample divisor and construct a polytope    ####
###########################################################

def ample_divisor(X):
    weights = [1 for _ in nef_cone(X).rays()]  # Positive weights (e.g., all ones)
    ampleray = sum(w * r for w, r in zip(weights, nef_cone(X).rays()))
    return ampleray

def poly(r,A):
    ieq=(matrix(A).transpose()).augment(matrix(r)).rows()
    return Polyhedron(ieqs=ieq)

def polytope_from_toric(X):
    H=unPic(X,ample_divisor(X))
    return poly(X.fan().rays(),H)

##############################################################
###  Check if a divisor is ample/movable/effective    ########
##############################################################

def is_ample(X,D):
    return nef_cone(X).relative_interior_contains(Pic(X,D))

def is_movable(X,D):
    return mov_cone(X).relative_interior_contains(Pic(X,D))

def is_effective(X,D):
    return eff_cone(X).contains(Pic(X,D))

####################################################
###################  Check Fanitude ################
####################################################

def is_Fano(X):
    if 'Polyhedra' in str(X.category()):
        X = ToricVariety(NormalFan(X))
    return   (-X.K()).is_ample() 

In [5]:
##########################################################################################
###################### Compute the secondary fan of the toric variety X ##################
##########################################################################################

######################################################
####### Remove duplicates from a list of cones #######
######################################################

def remove_duplicates(C):
    i = 0
    while i < len(C):
        j = i + 1
        while j < len(C):
            if C[i].is_equivalent(C[j]):  # Check if C[i] is equivalent to C[j]
                C.pop(j)  # Remove the duplicate cone
            else:
                j += 1
        i += 1

########################################################################
####### Remove cones containing other cones from a list of cones #######
########################################################################

def remove_containing(C):
    n = len(C)
    to_keep = set(range(n))  

    for i in range(n):
        if i not in to_keep: 
            continue

        for j in range(i + 1, n):
            if j not in to_keep:  
                continue

            Cint = C[i].intersection(C[j])
            if Cint.is_equivalent(C[j]):  
                to_keep.discard(i)
                break  
            elif Cint.is_equivalent(C[i]): 
                to_keep.discard(j)

    to_keep = sorted(to_keep)  
    C[:] = [C[i] for i in to_keep]  

#########################################################################################
####### Consider all the intersections of cones in two lists and purge the output #######
#########################################################################################

def cone_intersect(X,a,b):
    l = len(a)
    l1 = len(b)
    int=[]
    for j in range(l):  
        for i in range(l1):  
            coint = a[j].intersection(b[i])
            if coint.dim() == picard_number(X):  # Check if dimension matches
                int.append(coint)
    remove_duplicates(int)
    remove_containing(int)
    return int
    
##########################################################
####### Compute the secondary fan (rays and cones) #######
##########################################################

def starting_cones(X):
    L=X.Kaehler_cone().lattice()
    p = picard_number(X)
    A = Matrix(invariant_in_pic(X)).transpose()
    return [Cone([Matrix([[A[row][col] for col in col_indices] 
                           for row in row_indices]).column(i) 
                  for i in range(p)],lattice=L)
            for row_indices in combinations(range(A.nrows()), p)
            for col_indices in combinations(range(A.ncols()), p)
            if Matrix([[A[row][col] for col in col_indices] 
                       for row in row_indices]).determinant() != 0]

def check_lists(l1,l2):
    if len(l1) != len(l2):
        return False
    return all(any(cone1.is_equivalent(cone2) for cone2 in l2) for cone1 in l1)

def secondary_fan(X):
    co=co1=starting_cones(X)
    while True:
        new_cones = cone_intersect(X,co,co1)
        if check_lists(new_cones,co1) == True:
            break
        co1 = new_cones
        r = set()
    for cone in co1:
        r.update(cone.rays())
        rays=list(r)
    return rays,[[rays.index(ray) for ray in cone.rays()] for cone in co1],co1

###############################################################################
####### Find a divisor for every maximal dimensional chambers in Eff(X) #######
###############################################################################

def chambers(X):
    rays=secondary_fan(X)[0]
    cones=secondary_fan(X)[1]
    barycentric_rays = []
    for cone in cones:
        barycenter = sum(rays[i] for i in cone)
        barycentric_rays.append(unPic(X,barycenter))
    return barycentric_rays

###################################################################################
####### Find the polytopes for every maximal dimensional chambers in Eff(X) #######
###################################################################################

def models(X):
    mod=[]
    ch=chambers(X)
    for i in range(len(ch)):
        mod.append(poly(X.fan().rays(),ch[i]))
    return mod

###################################################################################
#######      Checks if a divisor is in the interior of a maximal chamber    #######
###################################################################################

def chamber_interior(Y,D):
    ray=Pic(Y,D)
    cones=secondary_fan(Y)[2]
    coint = [cone for cone in cones if cone.interior_contains(ray)]
    return len(coint) == 1    

In [6]:
################################################################################
###########  Construct a toric variety from primitive relations ################
################################################################################

#### The primitive relations must be given as a list of lists, for example
#### the Fano fourfold C3 in Batyrev's list is given by PR=[[1,1,1,0,0,0],[-1,-1,0,1,1,1]] 

def toric_from_pr(PR):
    PRA=matrix(PR)
    PRB=Matrix(PRA.transpose().kernel().basis()).transpose()
    rays=PRB.rows()
    # Extract positions of positive entries for each list
    primitive_collections = [set(i for i, value in enumerate(sublist) if value > 0) for sublist in PR]
    rays_set = set(range(0, PRB.nrows()))
    # Generate all subsets of size ncols(PRB)
    subsets = Subsets(rays_set, PRB.ncols())
    # Filter out subsets that contain any of the subsets in subsets_to_exclude
    cone_sets = [
    s for s in subsets if all(not exclude_set.issubset(s) for exclude_set in primitive_collections)]
    cones = [tuple(sorted(subset)) for subset in cone_sets]
    Fan_X=Fan(cones=cones,rays=rays)
    X=ToricVariety(Fan_X)
    return(X)

In [7]:
###########################################################################################
######################                                               ######################
######################              PROJECTIVE BUNDLES               ###################### 
######################                                               ######################
###########################################################################################

#### Preliminary step: find the rays of the fan of the projective bundle

############################################################
#### Rays associated with fibers of a projective bundle ####
############################################################

def fiber_rays(a,b):  #### Rays associated with fibers of a projective bundle
    FF = matrix(QQ,[[-1]*(a-1)] + list(identity_matrix(QQ,a-1)))
    return [list(ray) for ray in Matrix(QQ,a,b,0).augment(FF).rows()]

###################################
#### Rays coming from the base ####
###################################

def base_rays(r,A):
    a = A.nrows()  
    b = len(r[0])  
    raysf = fiber_rays(a,b)  
    raysb = []
    for n in range(len(r)):
        s = vector(r[n] + [0] * (a - 1))  
        for i in range(a):  
            s += A[i,n] * vector(raysf[i])
        raysb.append(list(s))  
    return raysb

#######################################
#### Rays of the projective bundle ####
#######################################

def proj_rays(r,A):  #### Rays of the fan of a projective bundle
    a=A.nrows()
    b=len(r[0])
    return base_rays(r,A) + fiber_rays(a,b)

#######################################################################################
####### Construct a toric projective bundle over X (toric variety or polytope)  #######
####### The bundle is given by a matrix M whose rows are the coordinates        #######
####### of the summands of the bundle on the invariant divisors                 #######
#######################################################################################

def projective_bundle(X,M): 
    if 'Polyhedra' in str(X.category()):
        X = ToricVariety(NormalFan(X))
    
    rays = [list(ray) for ray in (X.fan()).rays()]
    rank = M.nrows()
    FB = X.fan().cones()
    pbrays = proj_rays(rays, M)

    lc = []
    for i in range(1, len(FB)):  
        for cone in FB[i]:
            lc.append([rays.index(list(ray)) for ray in cone.rays()]) 
    
    elements = list(range(len(rays), len(rays) + rank))
    subsets = [list(subset) for subset in combinations(elements, rank - 1)]

    lcp = []
    for base_cone in lc:
        for fiber_subset in subsets:
            lcp.append(base_cone + fiber_subset)

    fan_cones = [Cone([pbrays[i] for i in cone]) for cone in lcp]
    pbfan = Fan(fan_cones, discard_faces=True)

    return ToricVariety(pbfan)